## 1、create_sql_query_chain
create_sql_query_chain，SQL查询链，是创建生成SQL查询的链，用于将 自然语言 转换成 数据库的SQL查询

In [3]:
from langchain_community.utilities import SQLDatabase

# 连接 MySQL 数据库
db_user = "root"
db_password = "xllzy123"  #根据自己的密码填写
db_host = "127.0.0.1"  #或者使用localhost
db_port = "3306"
db_name = "pet_shop"
# mysql+pymysql://用户名:密码@ip地址:端口号/数据库名
db = SQLDatabase.from_uri(
	f"mysql+pymysql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"
)
print("哪种数据库：", db.dialect)
print("获取数据表：", db.get_usable_table_names())
# 执行查询
res = db.run("SELECT count(*) FROM auth_user;")
print("查询结果：", res)


哪种数据库： mysql
获取数据表： ['accounts_userprofile', 'auth_group', 'auth_group_permissions', 'auth_permission', 'auth_user', 'auth_user_groups', 'auth_user_user_permissions', 'charts_soldmodel', 'charts_usermodel', 'commodity_commoditycategories', 'commodity_commodityinfos', 'customer_operation_useraddress', 'customer_operation_usercomment', 'customer_operation_userfav', 'customer_operation_userleavingmessage', 'django_admin_log', 'django_content_type', 'django_migrations', 'django_session', 'merchant_advertisement', 'trade_ordergoods', 'trade_orderinfos', 'trade_shoppingcart']
查询结果： [(11,)]


In [11]:
# 举例1
from langchain_openai import ChatOpenAI
from langchain.chains.sql_database.query import create_sql_query_chain
from langchain_community.utilities import SQLDatabase
import os
import dotenv

# 1、加载环境变量
dotenv.load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY1")
os.environ["OPENAI_BASE_URL"] = os.getenv("OPENAI_BASE_URL")

# 2、获取MySQL的数据库连接
db_user = "root"
db_password = "xllzy123"  #根据自己的密码填写
db_host = "127.0.0.1"  #或者使用localhost
db_port = "3306"
db_name = "pet_shop"
db = SQLDatabase.from_uri(
	f"mysql+pymysql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"
)

# 3.初始化语言模型
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
chain = create_sql_query_chain(llm, db)

# 4.使用链进行查询
question = "请问有多少个用户？"
response = chain.invoke({"question": question})
print("问题：", question)
print("回答：", response)

问题： 请问有多少个用户？
回答： SQLQuery: SELECT COUNT(`id`) AS `user_count` FROM `auth_user`;


## 2、create_stuff_documents_chain(了解)
create_stuff_documents_chain用于将 多个文档内容 合并成 单个长文本 的链式工具，并一次性传递给LLM处理（而不是分多次处理）。

适合场景：

- 保持上下文完整，适合需要全局理解所有文档内容的任务（如总结、问答）
- 适合处理 少量/中等长度文档 的场景。

In [12]:
# 举例1：多文档摘要
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.documents import Document

# 定义提示词模板
prompt = PromptTemplate.from_template("""
如下文档{docs}中说，香蕉是什么颜色的？
""")
# 创建链
llm = ChatOpenAI(model="gpt-4o-mini")
chain = create_stuff_documents_chain(llm, prompt, document_variable_name="docs")
# 文档输入
docs = [
	Document(
		page_content="苹果，学名Malus pumila Mill.，别称西洋苹果、柰，属于蔷薇科苹果属的植物。苹果是全球最广泛种植和销售的水果之一，具有悠久的栽培历史和广泛的分布范围。苹果的原始种群主要起源于中亚的天山山脉附近，尤其是现代哈萨克斯坦的阿拉木图地区，提供了所有现代苹果品种的基因库。苹果通过早期的贸易路线，如丝绸之路，从中亚向外扩散到全球各地。"
	),
	Document(
		page_content="香蕉是白色的水果，主要产自热带地区。"
	),
	Document(
		page_content="蓝莓是蓝色的浆果，含有抗氧化物质。"
	)
]
# 执行摘要
chain.invoke({"docs": docs})

'香蕉是黄色的水果。尽管文中提到香蕉是白色的，但实际上成熟的香蕉通常是黄色的，未成熟时可能呈现绿色。'